In [1]:
#Installations and imports
import pandas
!pip install nltk
import nltk
from pathlib import Path

import warnings
warnings.filterwarnings("ignore")

# data_path = Path("/mnt/f/workspace/DataCLUE/nips_expr/aclImdb")
data_path = Path("/home/jovyan/Data Mining Grad Project/ACL_IMDB")

nltk.download('stopwords')
from nltk.corpus import stopwords

stopWords = stopwords.words('english')

#install other important things
!pip install lightgbm
!pip install fasttext
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

# stopwords = set(Path("/mnt/f/workspace/DataCLUE/nips_expr/stopwords.txt").read_text().split("\n"))

  Using cached nltk-3.7-py3-none-any.whl (1.5 MB)
  Using cached regex-2022.4.24-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (763 kB)


[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


  Using cached lightgbm-3.3.2-py3-none-manylinux1_x86_64.whl (2.0 MB)
  Using cached fasttext-0.9.2-cp39-cp39-linux_x86_64.whl
  Using cached pybind11-2.9.2-py2.py3-none-any.whl (213 kB)


[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/jovyan/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [2]:
from tqdm import tqdm
def load_data(path):
    labels = []
    texts = []
    for file_path in tqdm((path / "neg").glob("*.txt")):
        labels.append(0)
        texts.append(file_path.read_text())
    
    for file_path in tqdm((path / "pos").glob("*.txt")):
        labels.append(1)
        texts.append(file_path.read_text())

    return texts, labels

all_train_texts, train_Y = load_data(data_path / "train")
all_test_texts, test_Y = load_data(data_path / "test")

1000it [00:00, 24468.14it/s]
12500it [00:09, 1379.11it/s]
12500it [00:09, 1359.04it/s]
12500it [00:09, 1359.46it/s]


In [3]:
from sklearn.feature_extraction.text import CountVectorizer
def data_quality(vectorizer, X, train_Y, cut_prop=1, filter_set = set()):
    from sklearn.feature_selection import f_classif, chi2
    F, pvalues_f = chi2(X, train_Y)
    
    last = int(len(F) * cut_prop)
    sorted_F = sorted(zip(vectorizer.get_feature_names(), F), key=lambda x: x[1], reverse=True)[:last]
    
    values = [value for name, value in sorted_F if name not in filter_set]
    # max_value, min_value = max(values), min(values)



    # values = [(value - min_value) / (max_value - min_value) for value in values]

    # print(values[:10])
    # print(values[-10:])

    return sum(values) / len(values)

In [4]:
def naive_bayes_classifier(train_x, train_y, test_x):  
    from sklearn.naive_bayes import MultinomialNB  
    model = MultinomialNB(alpha=0.01)  
    model.fit(train_x, train_y)
    return model.predict(test_x)

def svm_classifier(train_x, train_y, test_x):
    from sklearn.svm import SVC  
    model = SVC(kernel='rbf', probability=True, max_iter=50)
    model.fit(train_x, train_y)
    return model.predict(test_x)

def lgb_classifier(train_x, train_y, test_x):
    import lightgbm as lgb
    import numpy as np
    
    train_data = lgb.Dataset(train_x.astype('float32'), label=np.array(train_y, np.float32))
#     test_data = lgb.Dataset(test_x, label=test_y)
    
    params={
#         'learning_rate':0.1,
#         'lambda_l1':0.1,
#         'lambda_l2':0.2,
#         'max_depth':6,
#         'min_data_'
        'objective':'multiclass',
        'num_iteration': 200, 
        'num_class': 2,
    }
    
    model = lgb.train(params, train_data)
    return np.argmax(model.predict(test_x.astype('float32')), axis=1)

def fasttext_classifier(train_texts, train_y, test_texts, test_y, name):
    import fasttext

    model_path = data_path / f"fasttext.model.{name}.bin"
    if model_path.exists():
        classifier = fasttext.load_model(str(model_path))
    else:
        write_path = data_path / f"fasttext.data.train.{name}.txt"
        write_path.write_text("\n".join(
            f"{sentence}\t__label__{label}" for sentence, label in zip(train_texts, train_Y)
        ))

        classifier = fasttext.train_supervised(str(write_path), lr=0.1, word_ngrams=3, bucket=2000000, label_prefix="__label__")
        classifier.save_model(str(model_path))
        
    labels, _ = classifier.predict(test_texts)
    y_pred = [int(label[0][-1]) for label in labels]
    return y_pred

def knn_classifier(train_X, train_Y, test_X):
    from sklearn.neighbors import KNeighborsClassifier
    model = KNeighborsClassifier(n_neighbors=3)  
    model.fit(train_X, train_Y)
    return [4]
    return model.predict(test_X)

def textcnn(train_x, train_y):
    from deepclassifier.models import TextCNN
    from deepclassifier.trainers import Trainer
    import torch.optim as optim
    
    model = TextCNN(embedding_dim=300, dropout_rate=0.2, num_class=2)
    optimizer = optim.Adam(model.parameters())
    loss_fn = nn.CrossEntropyLoss()


In [5]:
model_mapper = {
    "nb": naive_bayes_classifier,
    "svm": svm_classifier,
    "lgb": lgb_classifier,
    "knn": knn_classifier,
    "fasttext": None,
    "textcnn": None,
}

def train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, model_name):
    if model_name not in model_mapper:
        y_pred = fasttext_classifier(train_texts, train_Y, test_texts, test_Y, model_name)
    else:
        y_pred = model_mapper[model_name](train_X, train_Y, test_X)
    from sklearn.metrics import accuracy_score
    return accuracy_score(test_Y, y_pred)

# No Operations_Normal

In [6]:
vectorizer = CountVectorizer(lowercase=False)
train_X = vectorizer.fit_transform(all_train_texts)


from sklearn.feature_selection import f_classif, chi2
F, pvalues_f = chi2(train_X, train_Y)

sorted_F = sorted(zip(vectorizer.get_feature_names(), F), key=lambda x: x[1], reverse=True)

In [7]:
all_train_texts[0]

'and rent a GOOD horror movie. It\'s like the writer had never seen a horror movie before and didn\'t realize every single thing he wrote was clichéd and hackneyed and has been parodied to perfection in movies like "Scream" and "Scary Movie".<br /><br />In between the scary bits is the most BANAL and BORING dialog ever written. Stupid "we\'re going to the prom" junk. I wanted to claw my ears off. Honestly, "The Hills" has better dialog.<br /><br />There really was no need to make this movie. Leading lady is uninteresting and I kept thinking "Her? Really? Guy is obsessed with her? Really?" <br /><br />All the characters act in stupid ways, including the police. (Cover the place in teams of 2! Front and back! Not one sleepy cop sitting in his car with the window rolled down just waiting for his throat to be slashed.) <br /><br />The serial killer just swans about murdering everyone he wants without the least bit of problem. No resistance from victims (or doors). Nobody has any protection

In [8]:
vectorizer = CountVectorizer(lowercase=False)
train_X = vectorizer.fit_transform(all_train_texts)
test_X = vectorizer.transform(all_test_texts)

data_quality(vectorizer, train_X, train_Y)

1.7865934961112424

In [9]:
train_model(all_train_texts, train_X, train_Y, all_test_texts, test_X, test_Y, "nb")

0.62976

In [10]:
train_model(all_train_texts, train_X, train_Y, all_test_texts, test_X, test_Y, "svm")

0.51788

In [11]:
train_model(all_train_texts, train_X, train_Y, all_test_texts, test_X, test_Y, "lgb")

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 3.627349 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 33533
[LightGBM] [Info] Number of data points in the train set: 13500, number of used features: 8978
[LightGBM] [Info] Start training from score -2.602690
[LightGBM] [Info] Start training from score -0.076961


0.65252

In [12]:
train_model(all_train_texts, train_X, train_Y, all_test_texts, test_X, test_Y, "normal")

Read 3M words
Number of words:  187892
Number of labels: 2
Progress: 100.0% words/sec/thread:  195786 lr:  0.000000 avg.loss:  0.239153 ETA:   0h 0m 0s


0.5

In [13]:
# train_model(all_train_texts, train_X, train_Y, all_test_texts, test_X, test_Y, "knn")

# LowerCase

In [14]:
from sklearn.feature_extraction.text import CountVectorizer

train_texts = [sentence.lower() for sentence in all_train_texts]
test_texts = [sentence.lower() for sentence in all_test_texts]

vectorizer = CountVectorizer(lowercase=True)
train_X = vectorizer.fit_transform(train_texts)
test_X = vectorizer.transform(test_texts)

data_quality(vectorizer, train_X, train_Y)

1.8859260771459267

In [15]:
train_texts[0]

'and rent a good horror movie. it\'s like the writer had never seen a horror movie before and didn\'t realize every single thing he wrote was clichéd and hackneyed and has been parodied to perfection in movies like "scream" and "scary movie".<br /><br />in between the scary bits is the most banal and boring dialog ever written. stupid "we\'re going to the prom" junk. i wanted to claw my ears off. honestly, "the hills" has better dialog.<br /><br />there really was no need to make this movie. leading lady is uninteresting and i kept thinking "her? really? guy is obsessed with her? really?" <br /><br />all the characters act in stupid ways, including the police. (cover the place in teams of 2! front and back! not one sleepy cop sitting in his car with the window rolled down just waiting for his throat to be slashed.) <br /><br />the serial killer just swans about murdering everyone he wants without the least bit of problem. no resistance from victims (or doors). nobody has any protection

In [16]:
train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "nb")

0.64124

In [17]:
train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "svm")

0.51444

In [18]:
train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "lgb")

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 3.364107 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 31942
[LightGBM] [Info] Number of data points in the train set: 13500, number of used features: 8363
[LightGBM] [Info] Start training from score -2.602690
[LightGBM] [Info] Start training from score -0.076961


0.65992

In [19]:
train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "lower")

Read 3M words
Number of words:  169944
Number of labels: 2
Progress: 100.0% words/sec/thread:  196580 lr:  0.000000 avg.loss:  0.217056 ETA:   0h 0m 0s 15.6% words/sec/thread:  198862 lr:  0.084394 avg.loss:  0.317472 ETA:   0h 0m22s 60.3% words/sec/thread:  197447 lr:  0.039683 avg.loss:  0.152661 ETA:   0h 0m10s


0.5

In [20]:
# train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "knn")

# Stem

In [21]:
import nltk
from nltk.stem import PorterStemmer

porter = PorterStemmer()

def stem_sentences(sentences: str) -> str:
    def stem_sentence(sentence):
        words = nltk.word_tokenize(sentence)
        return " ".join(
            porter.stem(word, to_lowercase=False) for word in words
        )
    return [stem_sentence(sentence) for sentence in tqdm(sentences)]

train_texts, test_texts = stem_sentences(all_train_texts), stem_sentences(all_test_texts)

100%|██████████| 25000/25000 [02:39<00:00, 156.60it/s]


In [22]:
train_texts[0]

"and rent a GOOD horror movi . It 's like the writer had never seen a horror movi befor and did n't realiz everi singl thing he wrote wa clichéd and hackney and ha been parodi to perfect in movi like `` Scream '' and `` Scari Movi '' . < br / > < br / > In between the scari bit is the most BANAL and BORING dialog ever written . Stupid `` we 're go to the prom '' junk . I want to claw my ear off . Honestli , `` The Hill '' ha better dialog. < br / > < br / > There realli wa no need to make thi movi . Lead ladi is uninterest and I kept think `` Her ? Realli ? Guy is obsess with her ? Realli ? '' < br / > < br / > All the charact act in stupid way , includ the polic . ( Cover the place in team of 2 ! Front and back ! Not one sleepi cop sit in hi car with the window roll down just wait for hi throat to be slash . ) < br / > < br / > The serial killer just swan about murder everyon he want without the least bit of problem . No resist from victim ( or door ) . Nobodi ha ani protect or the le

In [23]:
vectorizer = CountVectorizer(lowercase=True)
train_X = vectorizer.fit_transform(train_texts)
test_X = vectorizer.transform(test_texts)

data_quality(vectorizer, train_X, train_Y)

2.061223947072388

In [24]:
train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "nb")

0.66008

In [25]:
train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "svm")

0.51784

In [26]:
train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "lgb")

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 2.318744 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 27324
[LightGBM] [Info] Number of data points in the train set: 13500, number of used features: 6611
[LightGBM] [Info] Start training from score -2.602690
[LightGBM] [Info] Start training from score -0.076961


0.65352

In [27]:
train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "stem")

Read 3M words
Number of words:  73194
Number of labels: 2
Progress: 100.0% words/sec/thread:  212852 lr:  0.000000 avg.loss:  0.162846 ETA:   0h 0m 0s% words/sec/thread:  213267 lr:  0.049754 avg.loss:  0.148585 ETA:   0h 0m15s


0.5

In [28]:
# train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "knn")

# Lemmatization

In [29]:
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

In [30]:
def lemma_sentences(sentences: str) -> str:
    def lemma_sentence(sentence):
        return " ".join(
            wordnet_lemmatizer.lemmatize(word) for word in nltk.word_tokenize(sentence)
        )
    return [lemma_sentence(sentence) for sentence in tqdm(sentences)]

train_texts, test_texts = lemma_sentences(all_train_texts), lemma_sentences(all_test_texts)

100%|██████████| 25000/25000 [01:21<00:00, 305.72it/s]


In [31]:
train_texts[0]

"and rent a GOOD horror movie . It 's like the writer had never seen a horror movie before and did n't realize every single thing he wrote wa clichéd and hackneyed and ha been parodied to perfection in movie like `` Scream '' and `` Scary Movie '' . < br / > < br / > In between the scary bit is the most BANAL and BORING dialog ever written . Stupid `` we 're going to the prom '' junk . I wanted to claw my ear off . Honestly , `` The Hills '' ha better dialog. < br / > < br / > There really wa no need to make this movie . Leading lady is uninteresting and I kept thinking `` Her ? Really ? Guy is obsessed with her ? Really ? '' < br / > < br / > All the character act in stupid way , including the police . ( Cover the place in team of 2 ! Front and back ! Not one sleepy cop sitting in his car with the window rolled down just waiting for his throat to be slashed . ) < br / > < br / > The serial killer just swan about murdering everyone he want without the least bit of problem . No resistan

In [32]:
vectorizer = CountVectorizer(lowercase=True)
train_X = vectorizer.fit_transform(train_texts)
test_X = vectorizer.transform(test_texts)

data_quality(vectorizer, train_X, train_Y)

1.9242641955812305

In [33]:
train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "nb")

0.64548

In [34]:
train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "svm")

0.5192

In [35]:
train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "lgb")

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 2.911036 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 30154
[LightGBM] [Info] Number of data points in the train set: 13500, number of used features: 7744
[LightGBM] [Info] Start training from score -2.602690
[LightGBM] [Info] Start training from score -0.076961


0.65684

In [36]:
train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "lemma")

Read 3M words
Number of words:  88272
Number of labels: 2
Progress: 100.0% words/sec/thread:  210750 lr:  0.000000 avg.loss:  0.181554 ETA:   0h 0m 0s 50.7% words/sec/thread:  211966 lr:  0.049316 avg.loss:  0.157118 ETA:   0h 0m15s


0.5

In [37]:
# train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "knn")

# Remove Stop

In [38]:
def remove_stop_sentences(sentences: str) -> str:
    def remove_stop_sentence(sentence):
        return " ".join(
            word for word in nltk.word_tokenize(sentence) if word not in stopWords
        )
    return [remove_stop_sentence(sentence) for sentence in tqdm(sentences)]

train_texts, test_texts = remove_stop_sentences(all_train_texts), remove_stop_sentences(all_test_texts)

100%|██████████| 25000/25000 [01:02<00:00, 402.56it/s]


In [39]:
remove_stop_sentences([all_train_texts[0]])

100%|██████████| 1/1 [00:00<00:00, 168.98it/s]


["rent GOOD horror movie . It 's like writer never seen horror movie n't realize every single thing wrote clichéd hackneyed parodied perfection movies like `` Scream '' `` Scary Movie '' . < br / > < br / > In scary bits BANAL BORING dialog ever written . Stupid `` 're going prom '' junk . I wanted claw ears . Honestly , `` The Hills '' better dialog. < br / > < br / > There really need make movie . Leading lady uninteresting I kept thinking `` Her ? Really ? Guy obsessed ? Really ? '' < br / > < br / > All characters act stupid ways , including police . ( Cover place teams 2 ! Front back ! Not one sleepy cop sitting car window rolled waiting throat slashed . ) < br / > < br / > The serial killer swans murdering everyone wants without least bit problem . No resistance victims ( doors ) . Nobody protection least idea fighting back ( flipping security lock hotel room door ) . The people like mentally disabled sheep. < br / > < br / > By , 're gore fan , 'll disappointed . All killing kep

In [40]:
vectorizer = CountVectorizer(lowercase=True)
train_X = vectorizer.fit_transform(train_texts)
test_X = vectorizer.transform(test_texts)

data_quality(vectorizer, train_X, train_Y)

1.854712551428751

In [41]:
train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "nb")

0.62848

In [42]:
train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "svm")

0.5114

In [43]:
train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "lgb")

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 3.363173 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 30964
[LightGBM] [Info] Number of data points in the train set: 13500, number of used features: 8345
[LightGBM] [Info] Start training from score -2.602690
[LightGBM] [Info] Start training from score -0.076961


0.66568

In [44]:
train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "rem_stop")

Read 2M words
Number of words:  93535
Number of labels: 2
Progress: 100.0% words/sec/thread:  206976 lr:  0.000000 avg.loss:  0.195800 ETA:   0h 0m 0s


0.5

In [45]:
# train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "knn")

# f_classify threshold

In [46]:
vectorizer = CountVectorizer(lowercase=False)
train_X = vectorizer.fit_transform(all_train_texts)
test_X = vectorizer.transform(all_test_texts)

from sklearn.feature_selection import f_classif
F, pvalues_f = f_classif(train_X, train_Y)
value_names = sorted(zip(vectorizer.get_feature_names(), F), key=lambda x: x[1], reverse=True)
filter_names = [name for name, value in value_names[-int(len(value_names)*0.2):]]

def remove_words_sentences(sentences: str, removed_words: set) -> str:
    def remove_words_sentence(sentence):
        return " ".join(
            word for word in nltk.word_tokenize(sentence) if word not in removed_words
        )
    return [remove_words_sentence(sentence) for sentence in tqdm(sentences)]

train_texts, test_texts = remove_words_sentences(all_train_texts, set(filter_names)), remove_words_sentences(all_test_texts, set(filter_names))


vectorizer = CountVectorizer(lowercase=False)
train_X = vectorizer.fit_transform(train_texts)
test_X = vectorizer.transform(test_texts)

data_quality(vectorizer, train_X, train_Y)

100%|██████████| 25000/25000 [00:52<00:00, 478.89it/s]


2.148735931797523

In [47]:
train_texts[0]

"and rent a GOOD horror movie . It 's like the writer had never a horror movie and n't realize every single thing he wrote was clichéd and hackneyed and has been parodied to perfection in movies like `` Scream '' and `` Scary Movie '' . < br / > < br / > In between the scary bits is the most BANAL and BORING dialog ever written . Stupid `` 're going to the prom '' junk . I wanted to claw my ears off . Honestly , `` Hills '' has better dialog. < br / > < br / > There really was no need to make this movie . Leading is uninteresting and I kept thinking `` Her ? Really ? Guy is obsessed with ? Really ? '' < br / > < br / > All the characters act in stupid ways , including the police . ( Cover the in teams of 2 ! Front and back ! Not one sleepy sitting in his car with the window rolled down just waiting for his throat to be slashed . ) < br / > < br / > serial killer just swans about murdering everyone he wants without the least bit of problem . No resistance from victims ( or ) . Nobody ha

In [48]:
train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "nb")

0.62852

In [49]:
train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "svm")

0.51784

In [50]:
train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "lgb")

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 3.010197 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 29387
[LightGBM] [Info] Number of data points in the train set: 13500, number of used features: 7818
[LightGBM] [Info] Start training from score -2.602690
[LightGBM] [Info] Start training from score -0.076961


0.65548

In [51]:
train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "remove_f_thre")

Read 3M words
Number of words:  80452
Number of labels: 2
Progress: 100.0% words/sec/thread:  212769 lr:  0.000000 avg.loss:  0.138190 ETA:   0h 0m 0s


0.5

In [52]:
# train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "knn")

# CHI2_threshold
## Chi40

In [53]:
vectorizer = CountVectorizer(lowercase=False)
train_X = vectorizer.fit_transform(all_train_texts)

from sklearn.feature_selection import chi2
F, pvalues_f = chi2(train_X, train_Y)
value_names = sorted(zip(vectorizer.get_feature_names(), F), key=lambda x: x[1], reverse=True)
filter_names = [name for name, value in value_names[-int(len(value_names)*0.4):]]


In [54]:
remove_words_sentences([all_train_texts[0]], set(filter_names))

100%|██████████| 1/1 [00:00<00:00, 155.09it/s]


["and rent a GOOD horror movie . It 's like the writer had never a horror movie and n't realize every single thing he wrote was clichéd and hackneyed and has been parodied to perfection in movies like `` Scream '' and `` Scary Movie '' . < br / > < br / > In between the scary bits is the most BANAL and BORING dialog ever written . Stupid `` 're going to the prom '' junk . I wanted to claw my ears off . Honestly , `` Hills '' has better dialog. < br / > < br / > There really was no need to make this movie . Leading is uninteresting and I kept thinking `` Her ? Really ? Guy is obsessed with her ? Really ? '' < br / > < br / > All the characters act in stupid ways , including the police . ( Cover the in teams of 2 ! Front and back ! Not one sleepy cop sitting in his car with the window rolled down just waiting for his throat to be slashed . ) < br / > < br / > serial killer just swans about murdering everyone he wants without the least bit of problem . No resistance from victims ( or ) . 

In [55]:
train_texts, test_texts = remove_words_sentences(all_train_texts, set(filter_names)), remove_words_sentences(all_test_texts, set(filter_names))

vectorizer = CountVectorizer(lowercase=False)
train_X = vectorizer.fit_transform(train_texts)
test_X = vectorizer.transform(test_texts)

data_quality(vectorizer, train_X, train_Y)

100%|██████████| 25000/25000 [00:52<00:00, 475.58it/s]


2.7622077718188245

In [56]:
train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "nb")

0.62912

In [57]:
train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "svm")

0.5176

In [58]:
train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "lgb")

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 2.827376 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 29534
[LightGBM] [Info] Number of data points in the train set: 13500, number of used features: 7808
[LightGBM] [Info] Start training from score -2.602690
[LightGBM] [Info] Start training from score -0.076961


0.6522

In [59]:
train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "chi2_04")

Read 3M words
Number of words:  67115
Number of labels: 2
Progress: 100.0% words/sec/thread:  211138 lr:  0.000000 avg.loss:  0.253441 ETA:   0h 0m 0s 58.4% words/sec/thread:  211215 lr:  0.041608 avg.loss:  0.137040 ETA:   0h 0m11s 211170 lr:  0.016489 avg.loss:  0.267469 ETA:   0h 0m 4s


0.5

In [60]:
# train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "knn")

# CHI2_02
## Chi20

In [61]:
vectorizer = CountVectorizer(lowercase=False)
train_X = vectorizer.fit_transform(all_train_texts)
test_X = vectorizer.transform(all_test_texts)

from sklearn.feature_selection import chi2
F, pvalues_f = chi2(train_X, train_Y)
value_names = sorted(zip(vectorizer.get_feature_names(), F), key=lambda x: x[1], reverse=True)
filter_names = [name for name, value in value_names[-int(len(value_names)*0.2):]]

data_quality(vectorizer, train_X, train_Y, 1, set(filter_names))

2.215442112871815

In [62]:
train_texts, test_texts = remove_words_sentences(all_train_texts, set(filter_names)), remove_words_sentences(all_test_texts, set(filter_names))

vectorizer = CountVectorizer(lowercase=False)
train_X = vectorizer.fit_transform(train_texts)
test_X = vectorizer.transform(test_texts)
data_quality(vectorizer, train_X, train_Y)

100%|██████████| 25000/25000 [00:52<00:00, 479.55it/s]


2.1531971982226263

In [63]:
train_texts[0]

"and rent a GOOD horror movie . It 's like the writer had never a horror movie and n't realize every single thing he wrote was clichéd and hackneyed and has been parodied to perfection in movies like `` Scream '' and `` Scary Movie '' . < br / > < br / > In between the scary bits is the most BANAL and BORING dialog ever written . Stupid `` 're going to the prom '' junk . I wanted to claw my ears off . Honestly , `` Hills '' has better dialog. < br / > < br / > There really was no need to make this movie . Leading is uninteresting and I kept thinking `` Her ? Really ? Guy is obsessed with her ? Really ? '' < br / > < br / > All the characters act in stupid ways , including the police . ( Cover the in teams of 2 ! Front and back ! Not one sleepy cop sitting in his car with the window rolled down just waiting for his throat to be slashed . ) < br / > < br / > serial killer just swans about murdering everyone he wants without the least bit of problem . No resistance from victims ( or ) . N

In [64]:
train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "nb")

0.62896

In [65]:
train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "svm")

0.52032

In [66]:
train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "lgb")

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 2.902493 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 29921
[LightGBM] [Info] Number of data points in the train set: 13500, number of used features: 7926
[LightGBM] [Info] Start training from score -2.602690
[LightGBM] [Info] Start training from score -0.076961


0.65396

In [67]:
train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "chi2_02")

Read 3M words
Number of words:  80466
Number of labels: 2
Progress: 100.0% words/sec/thread:  211590 lr:  0.000000 avg.loss:  0.200390 ETA:   0h 0m 0ss


0.49996

In [68]:
# train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "knn")

In [69]:
# from sklearn.neighbors import KNeighborsClassifier
# neigh = KNeighborsClassifier(n_neighbors=3)  
# neigh.fit(train_X, train_Y)

In [ ]:
# result = neigh.predict(test_X)